<a href="https://colab.research.google.com/github/seshu-akkineni/AI-ML/blob/main/ImageEncryption.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
pip install pillow cryptography


In [3]:
from cryptography.hazmat.primitives.ciphers import Cipher, algorithms, modes
from cryptography.hazmat.primitives.kdf.pbkdf2 import PBKDF2HMAC
from cryptography.hazmat.primitives import hashes
from cryptography.hazmat.backends import default_backend
from PIL import Image
import os
import base64
import io

def encrypt_image(image_path, password):
    # Read the image
    with open("/content/samplee.jpg", 'rb') as image_file:
        image_data = image_file.read()

    # Generate a random salt
    salt = os.urandom(16)

    # Derive a key from the password
    kdf = PBKDF2HMAC(
        algorithm=hashes.SHA256(),
        length=32,
        salt=salt,
        iterations=100000,
        backend=default_backend()
    )
    key = kdf.derive(password.encode())

    # Generate a random initialization vector
    iv = os.urandom(16)

    # Create the cipher object
    cipher = Cipher(algorithms.AES(key), modes.CFB(iv), backend=default_backend())
    encryptor = cipher.encryptor()

    # Encrypt the image data
    encrypted_data = encryptor.update(image_data) + encryptor.finalize()

    # Combine salt, iv, and encrypted data
    encrypted_image = salt + iv + encrypted_data

    # Save the encrypted image
    encrypted_image_path = image_path + '.enc'
    with open(encrypted_image_path, 'wb') as encrypted_file:
        encrypted_file.write(encrypted_image)

    print(f"Image encrypted and saved as {encrypted_image_path}")

# Example usage
encrypt_image('samplee.jpeg', 'your_password')


Image encrypted and saved as samplee.jpeg.enc


In [4]:
def decrypt_image(encrypted_image_path, password):
    # Read the encrypted image
    with open("/content/samplee.jpeg.enc", 'rb') as encrypted_file:
        encrypted_image = encrypted_file.read()

    # Extract the salt, iv, and encrypted data
    salt = encrypted_image[:16]
    iv = encrypted_image[16:32]
    encrypted_data = encrypted_image[32:]

    # Derive the key from the password
    kdf = PBKDF2HMAC(
        algorithm=hashes.SHA256(),
        length=32,
        salt=salt,
        iterations=100000,
        backend=default_backend()
    )
    key = kdf.derive(password.encode())

    # Create the cipher object
    cipher = Cipher(algorithms.AES(key), modes.CFB(iv), backend=default_backend())
    decryptor = cipher.decryptor()

    # Decrypt the data
    decrypted_data = decryptor.update(encrypted_data) + decryptor.finalize()

    # Convert bytes back to image
    decrypted_image = Image.open(io.BytesIO(decrypted_data))

    # Save the decrypted image
    decrypted_image_path = encrypted_image_path.replace('.enc', '_decrypted.jpg')
    decrypted_image.save(decrypted_image_path)

    print(f"Image decrypted and saved as {decrypted_image_path}")

# Example usage
decrypt_image('example_image.jpg.enc', 'your_password')


Image decrypted and saved as example_image.jpg_decrypted.jpg
